# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f1f181223a0>
├── 'a' --> tensor([[-0.6729, -1.0808, -1.2457],
│                   [ 1.4491,  1.4587,  0.8920]])
└── 'x' --> <FastTreeValue 0x7f1f18122850>
    └── 'c' --> tensor([[ 0.5577,  1.0923, -0.7282, -0.1025],
                        [-0.7215, -1.0219, -1.9290,  0.6228],
                        [ 0.0292, -0.7678, -2.2441,  1.1048]])

In [4]:
t.a

tensor([[-0.6729, -1.0808, -1.2457],
        [ 1.4491,  1.4587,  0.8920]])

In [5]:
%timeit t.a

97.2 ns ± 1.35 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f1f181223a0>
├── 'a' --> tensor([[-0.4707, -0.4170, -1.1873],
│                   [ 0.3354, -0.7924,  0.2189]])
└── 'x' --> <FastTreeValue 0x7f1f18122850>
    └── 'c' --> tensor([[ 0.5577,  1.0923, -0.7282, -0.1025],
                        [-0.7215, -1.0219, -1.9290,  0.6228],
                        [ 0.0292, -0.7678, -2.2441,  1.1048]])

In [7]:
%timeit t.a = new_value

95.2 ns ± 1.99 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.6729, -1.0808, -1.2457],
               [ 1.4491,  1.4587,  0.8920]]),
    x: Batch(
           c: tensor([[ 0.5577,  1.0923, -0.7282, -0.1025],
                      [-0.7215, -1.0219, -1.9290,  0.6228],
                      [ 0.0292, -0.7678, -2.2441,  1.1048]]),
       ),
)

In [10]:
b.a

tensor([[-0.6729, -1.0808, -1.2457],
        [ 1.4491,  1.4587,  0.8920]])

In [11]:
%timeit b.a

79.4 ns ± 0.15 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.3519, -1.2807, -0.7876],
               [ 2.0563, -0.7730,  0.8072]]),
    x: Batch(
           c: tensor([[ 0.5577,  1.0923, -0.7282, -0.1025],
                      [-0.7215, -1.0219, -1.9290,  0.6228],
                      [ 0.0292, -0.7678, -2.2441,  1.1048]]),
       ),
)

In [13]:
%timeit b.a = new_value

660 ns ± 12.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.08 µs ± 25.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.1 µs ± 370 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

184 µs ± 5.01 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

187 µs ± 10.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f1e425f5760>
├── 'a' --> tensor([[[-0.6729, -1.0808, -1.2457],
│                    [ 1.4491,  1.4587,  0.8920]],
│           
│                   [[-0.6729, -1.0808, -1.2457],
│                    [ 1.4491,  1.4587,  0.8920]],
│           
│                   [[-0.6729, -1.0808, -1.2457],
│                    [ 1.4491,  1.4587,  0.8920]],
│           
│                   [[-0.6729, -1.0808, -1.2457],
│                    [ 1.4491,  1.4587,  0.8920]],
│           
│                   [[-0.6729, -1.0808, -1.2457],
│                    [ 1.4491,  1.4587,  0.8920]],
│           
│                   [[-0.6729, -1.0808, -1.2457],
│                    [ 1.4491,  1.4587,  0.8920]],
│           
│                   [[-0.6729, -1.0808, -1.2457],
│                    [ 1.4491,  1.4587,  0.8920]],
│           
│                   [[-0.6729, -1.0808, -1.2457],
│                    [ 1.4491,  1.4587,  0.8920]]])
└── 'x' --> <FastTreeValue 0x7f1e425f51c0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

39.4 µs ± 377 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f1e3a1cedc0>
├── 'a' --> tensor([[-0.6729, -1.0808, -1.2457],
│                   [ 1.4491,  1.4587,  0.8920],
│                   [-0.6729, -1.0808, -1.2457],
│                   [ 1.4491,  1.4587,  0.8920],
│                   [-0.6729, -1.0808, -1.2457],
│                   [ 1.4491,  1.4587,  0.8920],
│                   [-0.6729, -1.0808, -1.2457],
│                   [ 1.4491,  1.4587,  0.8920],
│                   [-0.6729, -1.0808, -1.2457],
│                   [ 1.4491,  1.4587,  0.8920],
│                   [-0.6729, -1.0808, -1.2457],
│                   [ 1.4491,  1.4587,  0.8920],
│                   [-0.6729, -1.0808, -1.2457],
│                   [ 1.4491,  1.4587,  0.8920],
│                   [-0.6729, -1.0808, -1.2457],
│                   [ 1.4491,  1.4587,  0.8920]])
└── 'x' --> <FastTreeValue 0x7f1e3a1ceee0>
    └── 'c' --> tensor([[ 0.5577,  1.0923, -0.7282, -0.1025],
                        [-0.7215, -1.0219, -1.9290,  0.6228],
                 

In [23]:
%timeit t_cat(trees)

36.9 µs ± 116 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

74.9 µs ± 128 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.5577,  1.0923, -0.7282, -0.1025],
                       [-0.7215, -1.0219, -1.9290,  0.6228],
                       [ 0.0292, -0.7678, -2.2441,  1.1048]],
              
                      [[ 0.5577,  1.0923, -0.7282, -0.1025],
                       [-0.7215, -1.0219, -1.9290,  0.6228],
                       [ 0.0292, -0.7678, -2.2441,  1.1048]],
              
                      [[ 0.5577,  1.0923, -0.7282, -0.1025],
                       [-0.7215, -1.0219, -1.9290,  0.6228],
                       [ 0.0292, -0.7678, -2.2441,  1.1048]],
              
                      [[ 0.5577,  1.0923, -0.7282, -0.1025],
                       [-0.7215, -1.0219, -1.9290,  0.6228],
                       [ 0.0292, -0.7678, -2.2441,  1.1048]],
              
                      [[ 0.5577,  1.0923, -0.7282, -0.1025],
                       [-0.7215, -1.0219, -1.9290,  0.6228],
                       [ 0.0292, -0.7678, -2.2441,  1.1048]],

In [26]:
%timeit Batch.stack(batches)

101 µs ± 3.11 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.5577,  1.0923, -0.7282, -0.1025],
                      [-0.7215, -1.0219, -1.9290,  0.6228],
                      [ 0.0292, -0.7678, -2.2441,  1.1048],
                      [ 0.5577,  1.0923, -0.7282, -0.1025],
                      [-0.7215, -1.0219, -1.9290,  0.6228],
                      [ 0.0292, -0.7678, -2.2441,  1.1048],
                      [ 0.5577,  1.0923, -0.7282, -0.1025],
                      [-0.7215, -1.0219, -1.9290,  0.6228],
                      [ 0.0292, -0.7678, -2.2441,  1.1048],
                      [ 0.5577,  1.0923, -0.7282, -0.1025],
                      [-0.7215, -1.0219, -1.9290,  0.6228],
                      [ 0.0292, -0.7678, -2.2441,  1.1048],
                      [ 0.5577,  1.0923, -0.7282, -0.1025],
                      [-0.7215, -1.0219, -1.9290,  0.6228],
                      [ 0.0292, -0.7678, -2.2441,  1.1048],
                      [ 0.5577,  1.0923, -0.7282, -0.1025],
                   

In [28]:
%timeit Batch.cat(batches)

179 µs ± 294 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

490 µs ± 15 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
